In [2]:
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
import scipy
from sympy.physics.mechanics import *

In [13]:
o_x, o_z, o_x_dot, o_z_dot, theta, theta_dot, alpha, alpha_dot = 

m, J_x, J_y, J_z, g, m_pen, l_pen = sym.symbols('m, J_x, J_y, J_z, g, m_pen, l_pen')
dyn_o_x, dyn_o_z, dyn_alpha, dyn_theta = dynamicsymbols(r'o_x o_z \alpha \theta')
dyn_o_xd, dyn_o_zd, dyn_alphad, dyn_thetad = dynamicsymbols(r'o_x o_z \alpha \theta', 1)
dyn_o_xdd, dyn_o_zdd, dyn_alphadd, dyn_thetadd = dynamicsymbols(r'o_x o_z \alpha \theta', 2)
T = 1/2*m*(dyn_o_xd**2 + dyn_o_zd**2) + 1/2*J_y*(dyn_thetad**2) + 1/2*m_pen*(dyn_o_xd**2 + dyn_o_zd**2 + l_pen**2 * dyn_alphad**2 + 2*l_pen*dyn_alphad*(dyn_o_xd*sym.cos(dyn_alpha) - dyn_o_zd*sym.sin(dyn_alpha)))
dyn_f_z, dyn_tau_y = sym.symbols(r'f_z, \tau_y')
t = sym.Symbol('t')

Equations = sym.Matrix([[sym.simplify(sym.diff(sym.diff(T, dyn_o_xd), t) - sym.diff(T, dyn_o_x) - dyn_f_z*sym.sin(dyn_theta))],
             [sym.simplify(sym.diff(sym.diff(T, dyn_o_zd), t) - sym.diff(T, dyn_o_z) - dyn_f_z*sym.cos(dyn_theta) + m*g)],
             [sym.simplify(sym.diff(sym.diff(T, dyn_alphad), t) - sym.diff(T, dyn_alpha) - m_pen*l_pen*g*sym.sin(dyn_alpha))],
             [sym.simplify(sym.diff(sym.diff(T, dyn_thetad), t) - sym.diff(T, dyn_theta) - dyn_tau_y)]])

Equations[3,0]

1.0*J_y*Derivative(\theta(t), (t, 2)) - \tau_y

In [17]:
ox_ddot_with_alphaddot = sym.solve(Equations[0,0], dyn_o_xdd)[0]
oz_ddot_with_alphaddot = sym.solve(Equations[1,0], dyn_o_zdd)[0]
alpha_ddot_eqn = Equations[2].subs([(dyn_o_xdd, ox_ddot_with_alphaddot), (dyn_o_zdd, oz_ddot_with_alphaddot)])
alpha_ddot_soln = sym.simplify(sym.solve(alpha_ddot_eqn, dyn_alphadd)[0])
new_ox_ddot = sym.simplify(ox_ddot_with_alphaddot.subs(dyn_alphadd, alpha_ddot_soln))
new_oz_ddot = sym.simplify(oz_ddot_with_alphaddot.subs(dyn_alphadd, alpha_ddot_soln))
theta_ddot_eqn = sym.solve(Equations[3,0], dyn_thetadd)[0]
# new_ox_ddot = new_ox_ddot.subs([(dyn_f_z, f_z), (dyn_theta, theta), (dyn_alphad, alpha_dot)])
# new_ox_ddot = new_ox_ddot.subs(dyn_alpha, alpha)
# new_oz_ddot = new_oz_ddot.subs([(dyn_f_z, f_z), (dyn_theta, theta), (dyn_alphad, alpha_dot)])
# new_oz_ddot = new_oz_ddot.subs(dyn_alpha, alpha)

# alpha_ddot_soln = alpha_ddot_soln.subs([(dyn_alpha, alpha), (dyn_theta, theta), (dyn_f_z, f_z)])
theta_ddot_eqn

\tau_y/J_y